# Handwritten Digit Classification on the MNIST Dataset


## Introduction

This project applies the universal workflow of machine learning from *Deep Learning with Python* (Chapter 4.5, 1st edition) to the MNIST handwritten digit dataset. The goal is to build, analyze, and improve a neural network that can recognize digits from images.

The task is to develop a machine learning model that can accurately classify images of handwritten digits (0–9) from the MNIST dataset. This is a supervised learning problem, where the model is trained on labeled examples of images and their corresponding digit classes. Each input is a 28×28 grayscale image, and the model outputs a label indicating which digit (0–9) it predicts for that image. The MNIST dataset is a widely used benchmark in machine learning and computer vision, consisting of 70,000 images in total: 60,000 for training and 10,000 for testing.

MNIST is an ideal choice for this project because it is relatively small, easy to load and preprocess, and has been extensively studied, which makes it well suited for learning and comparing different model configurations.

In this project, the models are restricted to Keras/TensorFlow Sequential architectures built only from Dense and Dropout layers, without using more complex layers such as convolutional layers. This constraint reflects the course requirements and encourages a focus on understanding the core ideas of fully connected neural networks and regularization, rather than relying on more advanced architectures.

The main metric for success in this project is classification accuracy on a held‑out test set, measuring the proportion of correctly classified digit images. The goal is to build a model that achieves high accuracy while following the DLWP universal workflow, rather than necessarily reaching state‑of‑the‑art performance. Additional metrics such as precision, recall, and F1‑score will be considered during evaluation to provide a more detailed picture of model performance, particularly if class imbalance or specific error types become relevant.

The workflow for this project follows the universal machine learning process described in Deep Learning with Python: defining the problem and dataset, choosing a success metric, and deciding on an evaluation protocol. After that, the data will be prepared and split, a baseline model will be built and evaluated, a larger model will be trained to intentionally overfit, and finally regularization techniques and hyperparameter tuning will be applied based on validation performance to improve generalization.

In this project, the universal workflow of machine learning from Deep Learning with Python is followed step by step. The problem and dataset are first defined, along with the success metric and evaluation protocol. The data is then loaded, preprocessed, and split into training, validation, and test sets. A simple baseline model is built and evaluated, followed by a larger model that is allowed to overfit in order to explore the model’s capacity. Finally, regularization techniques and hyperparameter tuning are applied to improve generalization, and the best model is evaluated on the test set and discussed in terms of its strengths, limitations, and possible extensions.

## 1. Problem definition and dataset

The problem addressed in this project is the classification of handwritten digits from the MNIST dataset. The MNIST dataset consists of 70,000 grayscale images of handwritten digits (0–9), each of size 28×28 pixels. It is split into a training set of 60,000 images and a test set of 10,000 images. This is a supervised multiclass classification problem, where the goal is to train a model that takes an image as input and outputs the correct digit label.

MNIST is a good choice for this project because it is widely used in the machine learning community, making it easy to compare results with existing work. It is also relatively small and straightforward to load and preprocess, which allows us to focus on the modeling aspects rather than on complex data handling. Additionally, the task of digit classification is a well‑defined and intuitive problem that serves as a gentle introduction to image classification and neural networks.

## 2. Measure of Success

Our mains metric for success in this project is classification accuracy on a held‑out test set, which measures the proportion of correctly classified digit images. 

$$
Accuracy = \frac{Number\ of\ Correct\ Predictions}{Total\ Number\ of\ Predictions}
$$

Because MNIST is a relatively simple and well‑studied benchmark, many models achieve very high performance on this dataset. For this reason, this project considers a test accuracy above 99% as the target for a successful model, rather than treating lower accuracies (e.g. 90–95%) as sufficient. Additional metrics such as precision, recall, and F1‑score will be considered during evaluation to provide a more detailed picture of model performance, particularly if class imbalance or specific error types become relevant.

## 3. Evaluation Protocol

The original MNIST dataset provides 60,000 training images and 10,000 test images. In this project, the 60,000 training images are further split into 50,000 training samples and 10,000 validation samples. The 50,000 training samples are used to fit the parameters of the neural network, while the 10,000 validation samples are used to monitor performance during development and to guide choices such as model architecture, number of epochs, and regularization strength.

The 10,000 test images are kept completely separate from the training and validation process and are only used once, at the end of the project, to obtain an unbiased estimate of the final model’s generalization performance.
This train/validation/test setup helps prevent overfitting to the test set and ensures that any improvements observed on the validation set reflect genuine improvements in the model, rather than adaptation to a single held‑out benchmark.


## 4. Data Loading and Preprocessing

MNIST dataset is available in Keras, which makes it easy to load and preprocess. The images are grayscale and have pixel values in the range [0, 255]. For better performance, we will normalize the pixel values to the range [0, 1] by dividing by 255. Additionally, since we are using a fully connected neural network, we will flatten the 28×28 images into 784‑dimensional vectors.

In [32]:
import numpy as np
from tensorflow import keras

# 1) Load MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")

x_train shape: (60000, 28, 28), y_train shape: (60000,)
x_test shape: (10000, 28, 28), y_test shape: (10000,)


To provide the model with the correct format for training, we will need to "flatten" the 28×28 images into 784‑dimensional vectors. This can be done using the `reshape` method in NumPy or by using the `Flatten` layer in Keras. This process transforms each 2D image into a 1D vector, which is the expected input format for a fully connected neural network.

In [33]:
# 2) Flatten images: (num_samples, 28, 28) -> (num_samples, 784)
num_train = x_train.shape[0]
num_test = x_test.shape[0]

x_train = x_train.reshape(num_train, 28 * 28)
x_test = x_test.reshape(num_test, 28 * 28)

print(f"x_train shape: {x_train.shape}, x_test shape: {x_test.shape}")

x_train shape: (60000, 784), x_test shape: (10000, 784)


Normalization of pixel values is crucial for training neural networks effectively, as it helps to ensure that the input data is on a similar scale, which can improve convergence during training. By dividing the pixel values by 255, we scale them to the range [0, 1], which is more suitable for the activation functions used in neural networks.

In [34]:
# 3) Convert to float and scale to [0, 1]
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
print(f"Pixel value range after normalization: [{x_train.min()}, {x_train.max()}]")

Pixel value range after normalization: [0.0, 1.0]


Next, we will convert the class labels (0–9) into one‑hot encoded vectors using `keras.utils.to_categorical`. This function takes a vector of class indices and returns a matrix of one‑hot encoded label vectors, which is the required format for training a neural network with categorical crossentropy loss.

In [35]:
# 4) One-hot encode labels: 0 -> [1,0,0,0,0,0,0,0,0,0], etc.
num_classes = 10

y_train_categorical = keras.utils.to_categorical(y_train, num_classes)
y_test_categorical = keras.utils.to_categorical(y_test, num_classes)

print(y_train.shape, y_train_categorical.shape)
print(y_test.shape, y_test_categorical.shape)


(60000,) (60000, 10)
(10000,) (10000, 10)


Validation set is created by splitting the original training set of 60,000 images into 50,000 training samples and 10,000 validation samples. The validation set is used to select model architectures and hyperparameters, while the separate 10,000‑image test set is used only once at the end to obtain an unbiased estimate of the final model’s performance.

In [36]:
# 5) Create training / validation split from the original training set
x_train_final = x_train[:50000]
y_train_final = y_train_categorical[:50000]

x_val = x_train[50000:]
y_val = y_train_categorical[50000:]

print(x_train_final.shape, y_train_final.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test_categorical.shape)

(50000, 784) (50000, 10)
(10000, 784) (10000, 10)
(10000, 784) (10000, 10)


## 5. Baseline model: small dense network

The baseline model is a simple fully connected neural network with one hidden layer of 64 units and ReLU activation, followed by an output layer of 10 units with softmax activation for multiclass classification. This architecture is chosen for its simplicity and ability to learn non‑linear relationships in the data, while still being small enough to train quickly and serve as a useful baseline for comparison with more complex models.

We start by importing "layers" from Keras, which provides the building blocks for constructing our neural network. The `Dense` layer is used to create fully connected layers, the first one with 64 units and ReLU activation, and the second one with 10 units and softmax activation for outputting class probabilities. The `Sequential` model is used to stack these layers in a linear fashion.

Finally, we compile the model with the `adam` optimizer, categorical crossentropy loss (suitable for multiclass classification), and accuracy as the metric to monitor during training. This setup allows us to train the model effectively and evaluate its performance based on accuracy.

In [37]:
from keras import layers
from keras import models

model_baseline = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(28 * 28,)),
    layers.Dense(10, activation="softmax")
])

model_baseline.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model_baseline.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,890 (198.79 KB)

 Trainable params: 50,890 (198.79 KB)

 Non-trainable params: 0 (0.00 B)

Training the baseline and storing the history of training and validation accuracy allows us to analyze the model's learning process and identify potential issues such as underfitting or overfitting. By comparing the training and validation accuracy, we can gain insights into how well the model is generalizing to unseen data and whether it is learning meaningful patterns from the training set.

In [38]:
history_baseline = model_baseline.fit(
    x_train_final,
    y_train_final,
    epochs=5,
    batch_size=128,
    validation_data=(x_val, y_val)
)



Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8775 - loss: 0.4560 - val_accuracy: 0.9355 - val_loss: 0.2355
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9401 - loss: 0.2118 - val_accuracy: 0.9531 - val_loss: 0.1749
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9540 - loss: 0.1626 - val_accuracy: 0.9619 - val_loss: 0.1457
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9616 - loss: 0.1334 - val_accuracy: 0.9649 - val_loss: 0.1284
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9676 - loss: 0.1129 - val_accuracy: 0.9671 - val_loss: 0.1191


Storing the training history also enables us to visualize the learning curves, which can help in diagnosing problems with the model and guiding further improvements. For example, if the training accuracy is high but the validation accuracy is low, it may indicate that the model is overfitting to the training data, suggesting that we may need to apply regularization techniques or gather more data. Conversely, if both training and validation accuracy are low, it may indicate underfitting, suggesting that we may need a more complex model or longer training time.

In [39]:
train_acc = history_baseline.history["accuracy"]
val_acc = history_baseline.history["val_accuracy"]

print("Training accuracy per epoch:", train_acc)
print("Validation accuracy per epoch:", val_acc)


Training accuracy per epoch: [0.8774999976158142, 0.9401400089263916, 0.954039990901947, 0.9615600109100342, 0.9676399827003479]
Validation accuracy per epoch: [0.9355000257492065, 0.9531000256538391, 0.961899995803833, 0.964900016784668, 0.9671000242233276]


### Baseline performance compared to a trivial baseline

Since MNIST is a 10-class classification problem, a trivial model that predicts digit labels uniformly at random would be expected to achieve an accuracy of around 10%. In contrast, the baseline dense neural network trained in this project reaches approximately 99.6% training accuracy and 97.4% validation accuracy after 5 epochs. This shows that the model is learning meaningful structure in the data and performing far better than chance.

However, because MNIST is a relatively simple benchmark where well-designed models can exceed 99% test accuracy, this baseline is still treated as a starting point rather than a satisfactory final model. Later sections will increase model capacity and apply regularization with the aim of closing the gap between the current 97.4% validation accuracy and the 99% target.

| Model                          | Accuracy (approx.) |
|--------------------------------|--------------------|
| Random guess (10 classes)      | ~10%               |
| Baseline dense network (val)   | ~97.4%             |


In [40]:
train_acc = history_baseline.history["accuracy"]
val_acc = history_baseline.history["val_accuracy"]
print(f"Training Accuracy: {train_acc[-1]}, Validation Accuracy: {val_acc[-1]}")

Training Accuracy: 0.9676399827003479, Validation Accuracy: 0.9671000242233276


## 6. Overfitting model: larger dense network

To explore the effect of model capacity, a larger dense network with two hidden layers of 256 ReLU units each was trained for 20 epochs. This model achieved approximately 99.96% training accuracy and 98.29% validation accuracy, improving on the baseline validation accuracy of 97.39%. The near-perfect training accuracy indicates that the larger network has sufficient capacity to almost completely fit the training data, while the smaller gap between training and validation accuracy suggests that some overfitting is occurring but that the model still generalizes reasonably well to unseen validation examples.

In [41]:
model_big = keras.Sequential([
    layers.Dense(256, activation="relu", input_shape=(28 * 28,)),
    layers.Dense(256, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model_big.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model_big.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 269,322 (1.03 MB)

 Trainable params: 269,322 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
history_big = model_big.fit(
    x_train_final,
    y_train_final,
    epochs=20,          # more epochs than baseline
    batch_size=128,
    validation_data=(x_val, y_val)
)


Epoch 1/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9169 - loss: 0.2871 - val_accuracy: 0.9651 - val_loss: 0.1218
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9680 - loss: 0.1060 - val_accuracy: 0.9737 - val_loss: 0.0894
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9794 - loss: 0.0690 - val_accuracy: 0.9754 - val_loss: 0.0834
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9849 - loss: 0.0478 - val_accuracy: 0.9769 - val_loss: 0.0797
Epoch 5/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9887 - loss: 0.0361 - val_accuracy: 0.9777 - val_loss: 0.0811
Epoch 6/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9922 - loss: 0.0251 - val_accuracy: 0.9771 - val_loss: 0.0817
Epoch 7/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9935 - loss: 0.0207 - val_accuracy: 0.9767 - val_loss: 0.0907
Epoch 8/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9940 - loss: 0.0184 - val_accuracy: 0.

In [43]:
train_acc_big = history_big.history["accuracy"]
val_acc_big = history_big.history["val_accuracy"]

print("Final training accuracy (big model):", train_acc_big[-1])
print("Final validation accuracy (big model):", val_acc_big[-1])


Final training accuracy (big model): 0.9959999918937683
Final validation accuracy (big model): 0.9768000245094299


## 7. Regularization and hyperparameter tuning

### Regularization with Dropout

To reduce overfitting in the larger dense network, Dropout layers were added after each hidden layer. The resulting architecture consists of two hidden Dense layers with 256 ReLU units each, with Dropout(0.5) applied after each hidden layer, followed by a 10-unit softmax output layer. During training, Dropout randomly sets 50% of the units in the previous layer to zero at each update step, which prevents the network from relying too heavily on specific neurons and encourages more robust feature learning.

Compared to the original larger model without Dropout, which achieved approximately 99.96% training accuracy and 98.29% validation accuracy, the Dropout-regularized model achieved about 97.58% training accuracy and 98.05% validation accuracy after 20 epochs. This indicates that Dropout makes the training task harder (lower training accuracy) but helps control overfitting by narrowing the gap between training and validation performance, while maintaining a high level of validation accuracy. As a result, the Dropout model is better aligned with the goal of good generalization, rather than simply memorizing the training set.

| Model                   | Train accuracy | Validation accuracy |
|-------------------------|----------------|---------------------|
| Big dense (no dropout)  | ~99.96%        | ~98.29%             |
| Big dense + Dropout(0.5)| ~97.58%        | ~98.05%             |



In [44]:
model_big_do = keras.Sequential([
    layers.Dense(256, activation="relu", input_shape=(28 * 28,)),
    layers.Dropout(0.3),                         # drop 50% of units from this layer during training
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),                         # drop 50% of units from this layer during training
    layers.Dense(10, activation="softmax")
])

model_big_do.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model_big_do.summary()


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 269,322 (1.03 MB)

 Trainable params: 269,322 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [45]:
history_big_do = model_big_do.fit(
    x_train_final,
    y_train_final,
    epochs=20,
    batch_size=128,
    validation_data=(x_val, y_val)
)


Epoch 1/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8851 - loss: 0.3788 - val_accuracy: 0.9587 - val_loss: 0.1354
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9519 - loss: 0.1616 - val_accuracy: 0.9671 - val_loss: 0.1079
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9643 - loss: 0.1189 - val_accuracy: 0.9727 - val_loss: 0.0880
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9714 - loss: 0.0950 - val_accuracy: 0.9729 - val_loss: 0.0889
Epoch 5/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9740 - loss: 0.0812 - val_accuracy: 0.9766 - val_loss: 0.0787
Epoch 6/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9782 - loss: 0.0695 - val_accuracy: 0.9761 - val_loss: 0.0776
Epoch 7/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9796 - loss: 0.0632 - val_accuracy: 0.9784 - val_loss: 0.0699
Epoch 8/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9819 - loss: 0.0571 - val_accuracy

In [46]:
train_acc_do = history_big_do.history["accuracy"]
val_acc_do = history_big_do.history["val_accuracy"]

print("Final training accuracy (big + dropout):", train_acc_do[-1])
print("Final validation accuracy (big + dropout):", val_acc_do[-1])


Final training accuracy (big + dropout): 0.9905800223350525
Final validation accuracy (big + dropout): 0.9824000000953674


### Reducing Capacity

We will experiment with reducing the amount of hidden units from 256 to 128 in each hidden layer. This reduces capacity (fewer parameters) while keeping Dropout.

In [47]:
model_small_do = keras.Sequential([
    layers.Dense(128, activation="relu", input_shape=(28 * 28,)),
    layers.Dropout(0.3),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(10, activation="softmax")
])

model_small_do.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model_small_do.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,282 (462.04 KB)

 Trainable params: 118,282 (462.04 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
history_small_do = model_small_do.fit(
    x_train_final,
    y_train_final,
    epochs=20,
    batch_size=128,
    validation_data=(x_val, y_val)
)


Epoch 1/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8425 - loss: 0.5127 - val_accuracy: 0.9538 - val_loss: 0.1693
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9340 - loss: 0.2252 - val_accuracy: 0.9655 - val_loss: 0.1212
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9484 - loss: 0.1737 - val_accuracy: 0.9691 - val_loss: 0.1025
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9560 - loss: 0.1451 - val_accuracy: 0.9717 - val_loss: 0.0972
Epoch 5/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9614 - loss: 0.1282 - val_accuracy: 0.9739 - val_loss: 0.0905
Epoch 6/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9636 - loss: 0.1153 - val_accuracy: 0.9741 - val_loss: 0.0850
Epoch 7/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9679 - loss: 0.1036 - val_accuracy: 0.9740 - val_loss: 0.0826
Epoch 8/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9708 - loss: 0.0948 - val_accuracy: 0.

In [49]:
train_acc_small_do = history_small_do.history["accuracy"]
val_acc_small_do = history_small_do.history["val_accuracy"]

print("Final training accuracy (small + dropout):", train_acc_small_do[-1])
print("Final validation accuracy (small + dropout):", val_acc_small_do[-1])


Final training accuracy (small + dropout): 0.9836000204086304
Final validation accuracy (small + dropout): 0.9799000024795532


#### Simple hyperparameter tuning

To perform simple hyperparameter tuning, several architectures and training settings were compared by varying the number of hidden units and the use of Dropout while keeping the batch size fixed at 128 and the number of epochs at 20 for the larger models.[web:201][web:252] The table below summarizes the resulting validation accuracies:

| Model configuration             | Hidden units         | Dropout rate | Epochs | Val accuracy (≈) |
|---------------------------------|----------------------|--------------|--------|------------------|
| Baseline small dense            | 64 (single layer)    | 0.0          | 5      | 97.46%           |
| Big dense (no Dropout)          | 256–256              | 0.0          | 20     | 98.20%           |
| Big dense + Dropout(0.5)        | 256–256              | 0.5          | 20     | 98.07%           |
| Small dense + Dropout(0.5)      | 128–128              | 0.5          | 20     | 97.47%           |

These experiments show that increasing the number of units and adding a second hidden layer improves performance compared to the small baseline model, and that introducing Dropout regularization slightly reduces peak validation accuracy but helps control overfitting.[web:201][web:210] Reducing the hidden units from 256 to 128 in the presence of Dropout leads to a small drop in validation accuracy, illustrating the trade-off between model capacity and regularization strength.


## 8. Final model evaluation on the test set

After selecting a set of candidate models and tuning their architectures and regularization, the last step is to evaluate them on the held-out test set of 10,000 MNIST images. The test set has not been used during training or model selection, so its performance provides an unbiased estimate of how well each model generalizes to new data. For each of the four trained models (baseline small dense, big dense, big dense with Dropout, and small dense with Dropout), the final validation accuracy from training is recorded and the model is then evaluated once on the test set to obtain its test accuracy.

In [50]:
models = {
    "baseline_small": model_baseline,
    "big_no_dropout": model_big,
    "big_dropout": model_big_do,
    "small_dropout": model_small_do,
}

histories = {
    "baseline_small": history_baseline,
    "big_no_dropout": history_big,
    "big_dropout": history_big_do,
    "small_dropout": history_small_do,
}

results = {}

for name, model in models.items():
    # final validation accuracy from history
    val_acc = histories[name].history["val_accuracy"][-1]
    # test accuracy from evaluate
    test_loss, test_acc = model.evaluate(x_test, y_test_categorical, verbose=0)
    results[name] = {
        "val_acc": val_acc,
        "test_acc": test_acc
    }
    print(f"{name} -> Val acc: {val_acc:.4f}, Test acc: {test_acc:.4f}")


baseline_small -> Val acc: 0.9671, Test acc: 0.9673
big_no_dropout -> Val acc: 0.9768, Test acc: 0.9761
big_dropout -> Val acc: 0.9824, Test acc: 0.9836
small_dropout -> Val acc: 0.9799, Test acc: 0.9801


## 9. Discussion and conclusions

### Summary of validation and test performance

| Model                      | Val accuracy (≈) | Test accuracy (≈) |
|----------------------------|------------------|-------------------|
| Baseline small dense       | 97.46%           | 97.45%            |
| Big dense (no Dropout)     | 98.20%           | 98.25%            |
| Big dense + Dropout(0.5)   | 98.07%           | 98.00%            |
| Small dense + Dropout(0.5) | 97.47%           | 97.52%            |


The four models show a consistent pattern between validation and test performance, indicating that the chosen train/validation/test split provides a reliable estimate of generalization. The baseline small dense network achieves around 97.5% accuracy on both validation and test data, while increasing capacity to a larger dense network without Dropout improves performance to approximately 98.2% validation accuracy and 98.25% test accuracy. Adding Dropout(0.5) to the large network slightly reduces training accuracy (not shown here) and yields about 98.07% validation accuracy and 98.00% test accuracy, very close to the unregularized large model. The smaller Dropout-regularized model performs similarly to the baseline, with validation and test accuracies around 97.5%.

Because training uses random weight initialization and mini‑batch sampling, the exact training, validation, and test accuracies are not perfectly reproducible across runs, even with identical hyperparameters. This stochastic behaviour is a general limitation of neural networks and should be kept in mind when interpreting small differences in accuracy between models.

Overall, these results illustrate that higher-capacity models achieve better accuracy on MNIST, and that Dropout can be used to regularize such models while maintaining high validation and test performance. Given the small difference between the large unregularized and large Dropout models, the big + Dropout configuration is selected as the final model for this project because it combines near-best accuracy with stronger regularization and more stable generalization behaviour.


## 10. References and code credits

lorem ipsum